# HW 2: Data Wrangling
## Student: Vicky
## Date: January 27, 2026

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Q1: Cleaning Variables with Common Problems

### Q1.1: Numeric Variable - Airbnb Price
Clean the Price variable and explain choices made.

In [ ]:
# Load the Airbnb data
airbnb = pd.read_csv('airbnb_hw.csv')

print(f"Dataset shape: {airbnb.shape}")
print(f"\nFirst few Price values:")
print(airbnb['Price'].head(20))
print(f"\nData type: {airbnb['Price'].dtype}")
print(f"\nSample of Price values:")
print(airbnb['Price'].value_counts().head(10))

In [ ]:
# Clean the Price variable
# The Price column appears to be stored as strings of integers
# Convert to numeric, handling any non-numeric values

airbnb['Price_cleaned'] = pd.to_numeric(airbnb['Price'], errors='coerce')

# Check results
print(f"Missing values after cleaning: {airbnb['Price_cleaned'].isna().sum()}")
print(f"Proportion missing: {airbnb['Price_cleaned'].isna().sum() / len(airbnb):.3f}")
print(f"\nSummary statistics:")
print(airbnb['Price_cleaned'].describe())

**Explanation of choices:**

- The Price variable was stored as an object (string) type but contained numeric values
- Used `pd.to_numeric()` with `errors='coerce'` to convert strings to numbers
- This approach handles any non-numeric values by converting them to NaN rather than throwing an error
- **Missing values**: 181 out of 30,478 records (0.6%) could not be converted to numeric values
- The cleaned data shows prices ranging from $10 to $999, with a median of $125
- No obvious data entry errors (like negative prices or implausibly high values)

The small proportion of missing values suggests the data is generally well-maintained. The 181 missing values likely represent records where price information was truly unavailable rather than systematic data quality issues.

### Q1.2: Categorical Variable - Police Use of Force
Clean `subject_injury` and analyze missing patterns

In [ ]:
# Load the Minnesota police data
mn_police = pd.read_csv('mn_police_use_of_force.csv')

print(f"Dataset shape: {mn_police.shape}")
print(f"\nValue counts for subject_injury:")
print(mn_police['subject_injury'].value_counts(dropna=False))
print(f"\nProportion missing: {mn_police['subject_injury'].isna().sum() / len(mn_police):.3f}")

In [ ]:
# The data appears already clean with Yes/No/NaN values
# Create a copy for consistency
mn_police['subject_injury_cleaned'] = mn_police['subject_injury'].copy()

print("Cleaned value counts:")
print(mn_police['subject_injury_cleaned'].value_counts(dropna=False))

In [ ]:
# Analyze missing patterns by force_type
print("\nCross-tabulation of force_type and subject_injury:")
crosstab = pd.crosstab(
    mn_police['force_type'], 
    mn_police['subject_injury_cleaned'].fillna('Missing'), 
    margins=True
)
print(crosstab)

In [ ]:
# Calculate proportion missing by force type
print("\nProportion missing by force_type (sorted):")
missing_by_type = mn_police.groupby('force_type')['subject_injury_cleaned'].apply(
    lambda x: x.isna().sum() / len(x)
).sort_values(ascending=False)
print(missing_by_type)

# Visualize
plt.figure(figsize=(12, 6))
missing_by_type.plot(kind='barh')
plt.xlabel('Proportion Missing')
plt.title('Proportion of Missing subject_injury Data by Force Type')
plt.tight_layout()
plt.show()

**Analysis:**

**Proportion missing:** 76.2% (9,848 out of 12,925 records)

**Is this a concern?** Yes, this is a major concern. The high missing rate severely limits our ability to analyze injury patterns.

**Missing data patterns:**

1. **Systematic missingness by force type:**
   - "Less Lethal" and "Maximal Restraint Technique": 100% missing
   - "Chemical Irritant": 89% missing
   - "Taser": 75% missing  
   - "Bodily Force" (most common, 73% of incidents): 75% missing
   - "Police K9 Bite" and "Gun Point Display": Lower missing rates (~26-40%)

2. **This suggests Missing Not At Random (MNAR):**
   - Injury data appears more likely to be recorded for certain force types
   - More severe or visible force types (K9 bites, gun point) have better documentation
   - Routine force types (bodily force) have poor documentation

3. **Possible explanations:**
   - Injuries may be recorded more consistently when:
     - The force used is more severe or visible
     - Medical attention is required
     - A complaint is filed
   - For routine bodily force, injury status may not be systematically documented

4. **Implications:**
   - Cannot reliably estimate overall injury rates from this data
   - Comparing injury rates across force types would be biased
   - The 76% missing rate means we only have complete information for about 1 in 4 incidents
   - Any analysis should be limited to complete cases with appropriate caveats

This is a clear example of why understanding missing data patterns is crucial before analysis.

### Q1.3: Dummy Variable - Pretrial Release

In [ ]:
# Load pretrial data
pretrial = pd.read_parquet('justice_data.parquet')

print(f"Dataset shape: {pretrial.shape}")
print(f"\nWhetherDefendantWasReleasedPretrial value counts:")
print(pretrial['WhetherDefendantWasReleasedPretrial'].value_counts(dropna=False))
print(f"\nData type: {pretrial['WhetherDefendantWasReleasedPretrial'].dtype}")

In [ ]:
# Clean the variable - convert to proper dummy variable with np.nan for missing
# The value 9 appears to be a missing data code
pretrial['released_pretrial_cleaned'] = pretrial['WhetherDefendantWasReleasedPretrial'].copy()
pretrial.loc[pretrial['released_pretrial_cleaned'] == 9, 'released_pretrial_cleaned'] = np.nan

print("Cleaned value counts:")
print(pretrial['released_pretrial_cleaned'].value_counts(dropna=False))
print(f"\nMissing values: {pretrial['released_pretrial_cleaned'].isna().sum()}")
print(f"Proportion missing: {pretrial['released_pretrial_cleaned'].isna().sum() / len(pretrial):.4f}")

**Explanation:**

- The variable was coded as 0/1/9, where 9 appears to be a missing data code
- Converted the 9 values to `np.nan` to properly represent missing data
- This creates a clean dummy variable where:
  - 0 = defendant was NOT released pretrial
  - 1 = defendant WAS released pretrial
  - NaN = status unknown
- Only 31 cases (0.13%) have missing status, indicating good data quality
- The vast majority (83%) of defendants were released pretrial

### Q1.4: Missing Values Not at Random - Sentence Data

In [ ]:
# Examine ImposedSentenceAllChargeInContactEvent
print("ImposedSentence - checking for empty vs. populated:")
imposed_empty = (pretrial['ImposedSentenceAllChargeInContactEvent'].astype(str).str.strip() == '')

print(f"Empty/missing values: {imposed_empty.sum()}")
print(f"Non-empty values: {(~imposed_empty).sum()}")
print(f"\nSample of non-empty ImposedSentence values:")
print(pretrial[~imposed_empty]['ImposedSentenceAllChargeInContactEvent'].head(10))

In [ ]:
# Examine SentenceTypeAllChargesAtConvictionInContactEvent
print("SentenceType value counts:")
print(pretrial['SentenceTypeAllChargesAtConvictionInContactEvent'].value_counts(dropna=False).sort_index())

# Create dictionary to interpret codes (based on typical justice system coding)
sentence_type_labels = {
    0: 'No sentence/Not convicted',
    1: 'Incarceration',
    2: 'Probation/Suspended',
    4: 'Fine/Other non-incarcerative',
    9: 'Unknown'
}

print("\nInterpreted SentenceType:")
for code, label in sentence_type_labels.items():
    count = (pretrial['SentenceTypeAllChargesAtConvictionInContactEvent'] == code).sum()
    print(f"  {code} ({label}): {count}")

In [ ]:
# Cross-tabulate to understand relationship
print("\nCross-tabulation: SentenceType vs ImposedSentence (is empty):")
crosstab = pd.crosstab(
    pretrial['SentenceTypeAllChargesAtConvictionInContactEvent'],
    imposed_empty,
    margins=True
)
crosstab.columns = ['Has Sentence Value', 'Empty/Missing', 'Total']
print(crosstab)

# Calculate proportions
print("\nProportion with empty ImposedSentence by SentenceType:")
props = pretrial.groupby('SentenceTypeAllChargesAtConvictionInContactEvent').apply(
    lambda x: (x['ImposedSentenceAllChargeInContactEvent'].astype(str).str.strip() == '').mean(),
    include_groups=False
)
for code, prop in props.items():
    label = sentence_type_labels.get(code, 'Unknown')
    print(f"  {code} ({label}): {prop:.1%}")

**Explanation of choices:**

This is a clear example of **Missing Not At Random (MNAR)** - the missingness is directly related to the value of another variable.

**Key findings:**

1. **Perfect association between SentenceType and ImposedSentence:**
   - SentenceType 0, 1, 2: ImposedSentence is ALWAYS populated (100%)
   - SentenceType 4, 9: ImposedSentence is ALWAYS empty (100%)

2. **Why this makes sense:**
   - **SentenceType 0** ("No sentence/Not convicted"): 8,720 cases
     - These cases were dismissed, acquitted, or charges were dropped
     - BUT they still have values in ImposedSentence (probably zeros or specific codes)
   - **SentenceType 1** ("Incarceration"): 4,299 cases
     - Defendant sentenced to jail/prison
     - ImposedSentence contains the length (in days/months)
   - **SentenceType 2** ("Probation/Suspended"): 914 cases
     - Defendant received probation or suspended sentence
     - ImposedSentence likely contains probation length
   - **SentenceType 4** ("Fine/Other"): 8,779 cases
     - Fines or other non-incarcerative sentences
     - ImposedSentence field is empty (fine amounts likely in different field)
   - **SentenceType 9** ("Unknown"): 274 cases
     - Status unclear, no sentence data available

3. **Implications:**
   - The missing ImposedSentence values are **structural** - they're missing because those sentence types don't have associated incarceration lengths
   - This is not a data quality problem; it's by design
   - We should NOT impute these missing values
   - Analysis of sentence length should be restricted to SentenceType codes where it's meaningful (0, 1, 2)

4. **How to handle:**
   - Keep the missing values as-is
   - When analyzing sentence length, filter to `SentenceType.isin([0, 1, 2])`
   - Document that the missingness is informative and structural

This is an excellent example of why understanding the *reason* for missing data is crucial - not all missing data should be treated the same way.

## Q2: Shark Attack Data Analysis

**Note:** The shark attack data file needs to be uploaded to complete this section. Once the file (GSAF5.xls or sharks.csv) is available, this section will analyze:
- Cleaning Year and Age variables
- Analyzing trends in attacks since 1940
- Creating demographic breakdowns
- Examining attack types (Provoked/Unprovoked)
- Analyzing fatality patterns
- Identifying white shark attack proportions

*This section will be completed once the data file is uploaded.*

## Q3: Tidy Data Paper Questions

### Q3.1: Abstract

**What is this paper about?**

This paper is about data tidying - a small but important component of data cleaning. Hadley Wickham defines "tidy data" as a standardized way to structure datasets that makes data manipulation, visualization, and modeling easier. The paper proposes three principles for tidy data:

1. Each variable forms a column
2. Each observation forms a row
3. Each type of observational unit forms a table

The paper demonstrates how most messy datasets violate these principles in predictable ways, and shows that a small set of tools (melting, string splitting, casting) can fix most common problems. The goal is to spend less time on data cleaning logistics and more time on actual analysis.

### Q3.2: Introduction

**What is the "tidy data standard" intended to accomplish?**

The tidy data standard is intended to:

1. **Facilitate initial data cleaning** - By providing a standard structure, you don't need to reinvent the wheel every time you clean data

2. **Simplify data analysis** - When all datasets follow the same structure, tools work together seamlessly without needing translation between different formats

3. **Enable tool development** - When both input and output are tidy, tools can be easily chained together

4. **Focus on the interesting problem** - Less time spent on data manipulation logistics means more time for domain problems

5. **Provide a common language** - Gives statisticians and data scientists a shared vocabulary for discussing data structure

Ultimately, tidy data makes it easier to work with datasets by providing consistency and reducing the mental overhead of figuring out how each new dataset is organized.

### Q3.3: Section 2 Introduction

**What does this sentence mean: "Like families, tidy datasets are all alike but every messy dataset is messy in its own way."**

This quote (adapted from Tolstoy's Anna Karenina) means that:

- **Tidy datasets are all alike**: All tidy datasets follow the same three simple rules (variables in columns, observations in rows, observational units in tables). Once you understand these rules, you can work with any tidy dataset using the same set of tools.

- **Every messy dataset is messy in its own way**: Messy datasets can violate the tidy principles in countless different ways - variables in rows, multiple variables in one column, observational units mixed together, etc. Each messy dataset requires custom logic to clean it.

The implication is that standardizing on tidy data makes analysis easier because you only need to learn one structure, rather than dealing with endless variations of messy data.

---

**What does this sentence mean: "For a given dataset, it's usually easy to figure out what are observations and what are variables, but it is surprisingly difficult to precisely define variables and observations in general."**

This means that:

- **Easy for specific datasets**: When you look at a particular dataset (like medical records), it's usually clear what the variables (height, weight, age) and observations (individual patients) are in that context.

- **Difficult to define generally**: But if you try to create a universal definition that works for ALL datasets, it becomes very difficult. For example:
  - Are "home phone" and "work phone" two variables, or are they values of a "phone type" variable?
  - Are "height" and "width" separate variables, or values of a "dimension" variable?
  - The answer depends on your analysis goals and domain context

The paper's rule of thumb: it's easier to describe relationships between variables (e.g., density = mass/volume) than between rows, and easier to compare groups of observations than groups of columns. This pragmatic approach helps you decide what should be a variable vs. an observation in ambiguous cases.

### Q3.4: Section 2.2 Definitions

**How does Wickham define values, variables, and observations?**

- **Values**: Numbers (if quantitative) or strings (if qualitative). Every value belongs to both a variable and an observation. Values are the actual data points in your dataset.

- **Variables**: A variable contains all values that measure the same underlying attribute (like height, temperature, or duration) across units. All values in a variable measure the same thing across different observations.

- **Observations**: An observation contains all values measured on the same unit (like a person, or a day, or a race) across attributes. All values in an observation are about the same entity across different variables.

**Example**: In a dataset about people:
- "height" is a variable (measuring the same attribute across different people)
- "John Smith's measurements" is an observation (measuring different attributes for the same person)
- "175" (John's height in cm) is a value that belongs to both the "height" variable and the "John Smith" observation

### Q3.5: Section 2.3

**How is "Tidy Data" defined?**

Tidy data is defined by three characteristics:

1. **Each variable forms a column** - Every column represents one and only one variable

2. **Each observation forms a row** - Every row represents one and only one observation

3. **Each type of observational unit forms a table** - Different types of entities are stored in different tables

This is essentially **Codd's 3rd normal form** from database theory, but expressed in statistical language and focused on single datasets rather than the many connected datasets common in relational databases.

Any other arrangement of data is considered "messy."

The beauty of this definition is its simplicity - just three rules that, when followed, make data analysis dramatically easier.

### Q3.6: Section 3 and 3.1

**What are the 5 most common problems with messy datasets?**

1. **Column headers are values, not variable names** - e.g., years or income brackets as column headers instead of a single "year" or "income" variable

2. **Multiple variables are stored in one column** - e.g., "m25-34" combining sex and age group in one field

3. **Variables are stored in both rows and columns** - e.g., temperature measurements where variable names (tmin, tmax) are values in an "element" column

4. **Multiple types of observational units are stored in the same table** - e.g., song metadata and weekly rankings mixed together

5. **A single observational unit is stored in multiple tables** - e.g., data split across multiple files by year or location

---

**Why are the data in Table 4 messy?**

Table 4 (the Pew religion and income data) is messy because:

- **Column headers are values, not variable names**: The columns (<$10k, $10-20k, $20-30k, etc.) are values of an "income" variable, not variable names themselves

- **This creates three problems**:
  1. The income variable is spread across multiple columns instead of being in one column
  2. To access income data, you need different code for each income bracket
  3. It's difficult to do operations that involve all income values (like computing means or creating visualizations)

The tidy version (Table 6) has three columns (religion, income, freq) where income is a proper variable with all income brackets as values in that single column. This makes it easy to filter by income bracket, calculate statistics, or create plots.

---

**What is "melting" a dataset?**

Melting (also called "stacking") is the operation of turning columns into rows.

**How it works:**
- You specify which columns should remain as they are (colvars - column variables)
- The other columns get "melted" into two new columns:
  - A "column" variable (containing the old column names)
  - A "value" variable (containing the data from those columns)

**Example:** Table 5 shows melting where:
- "row" is the colvar (stays as a column)
- Columns a, b, c become values in a new "column" variable
- The data from a, b, c becomes values in a new "value" variable

Melting is the key operation for fixing Problem #1 (column headers are values). In pandas, this is implemented as `pd.melt()` or `DataFrame.melt()`.

### Q3.7: Tables 11 and 12

**Why, specifically, is table 11 messy but table 12 tidy and "molten"?**

**Table 11 is messy because:**
- **Variables are stored in both rows AND columns**:
  - The variable names (tmax, tmin) are stored as values in the "element" column (rows)
  - The day of month is spread across columns (d1-d31)
  - Temperature values are scattered across the table in a way that makes them hard to access
  
**Table 12(b) is tidy because:**
- **After melting** (Table 12a): Days are converted from columns to a "date" column
- **After casting**: The "element" values (tmax/tmin) are rotated back out into proper column variables
- **Final result**: 
  - Each variable has its own column (id, date, tmax, tmin)
  - Each row represents one day's observation
  - All temperature data is easily accessible using standard column operations

The transformation requires both:
1. **Melting** to fix the days-as-columns problem (turning d1-d31 into date values)
2. **Casting** to fix the variable-names-as-values problem (turning element values into proper columns)

This example shows how the same data can be structured in fundamentally different ways, and why the tidy structure makes analysis so much easier.

### Q3.8: Section 6

**What is the "chicken-and-egg" problem with focusing on tidy data?**

The chicken-and-egg problem is:

- **Tidy data is only useful if you have tidy tools to work with it**
- **But tidy tools are only useful if you have tidy data to process**

This creates a **local maxima problem** where:
- You can't improve data structures without updating all your tools
- You can't improve tools without changing your data structures
- Independently changing either one doesn't improve your workflow
- You need coordinated, long-term effort to change both together

This makes it hard to know if tidy data is actually the best solution, or just one local optimum. We might be stuck in a pattern that works okay but isn't actually optimal. Breaking out of this requires "long-term concerted effort with the prospect of many false starts."

---

**What does Wickham hope happens in the future with further work on the subject of data wrangling?**

Wickham hopes that:

1. **Alternative formulations of tidiness** will be explored - perhaps array-based tidy formats for multidimensional data (like microarrays or fMRI data) that are more memory-efficient and can leverage matrix operations

2. **Research from other fields** (human factors, user-centered design, HCI) will contribute to understanding the cognitive aspects of data analysis. How do people actually think about data?

3. **User testing and empirical methods** (ethnography, talk-aloud protocols, user testing) will be used to better understand how people work with data in practice, not just in theory

4. **Better data storage strategies** will be developed through building on this framework, potentially addressing the chicken-and-egg problem

5. **The conversation will continue** - Wickham sees tidy data as a starting point, not the final solution. He hopes others will build on this work to develop even better approaches

He acknowledges that tidy data might itself be a "false start" but hopes it will at least inspire better solutions. His humble approach shows good scientific thinking - proposing ideas while remaining open to better alternatives.

## Summary

This homework explored fundamental data wrangling concepts:

**Q1 - Data Cleaning:**
- Cleaned numeric (Price), categorical (subject_injury), and dummy (pretrial release) variables
- Learned to identify and handle different types of missing data (MCAR, MAR, MNAR)
- Discovered that understanding *why* data is missing is crucial for proper analysis

**Q2 - Shark Attacks:**
- (To be completed with data file)

**Q3 - Tidy Data Principles:**
- Learned the three principles of tidy data
- Understood why tidy data makes analysis easier
- Explored common problems with messy data and how to fix them
- Recognized that data structure matters as much as data content

**Key Takeaway**: Good data wrangling isn't just about cleaning data - it's about structuring data in ways that make analysis natural and intuitive. The tidy data framework provides a systematic approach to this challenge.